In [19]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import mmap
import random
import pickle
import argparse

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

block_size = 128
batch_size = 32
n_embd = 256
n_layer = 4
n_head = 4
n_layer = 4
dropout = 0.2


max_iters = 1000
learning_rate = 3e-4
eval_iters = 250

# device side assert trigger errors
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'




# batch_size = args.batch_size # to use the batch_size cmd arg -> python file_name.py -batch_size 32


cuda


In [20]:
# with open('wizard-of-oz.txt', 'r', encoding='utf-8') as f:
#     text = f.read()

# chars = sorted(set(text))
# print(chars)

# vocab_size = len(chars)

In [21]:
import re

# Read the text file
with open('wizard-of-oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# Tokenize text into words and special characters
tokens = re.findall(r'\w+|[^\w\s]', text, re.UNICODE)

# Get unique tokens and sort them
unique_tokens = sorted(set(tokens))
print(unique_tokens)

# Calculate vocabulary size
vocab_size = len(unique_tokens)
print(f"Vocabulary size: {vocab_size}")





['!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '1', '10', '106', '11', '12', '120', '13', '131', '14', '142', '15', '16', '160', '17', '172', '18', '187', '19', '1908', '2', '20', '203', '217', '23', '231', '240', '251', '3', '4', '41', '5', '55', '6', '64', '7', '77', '8', '88', '9', '95', ':', ';', '?', 'A', 'AGAIN', 'AIR', 'ALL', 'AND', 'ANOTHER', 'ARE', 'ARRIVAL', 'AUTHOR', 'About', 'Accuser', 'Adjustable', 'After', 'Afterward', 'Ah', 'Ahem', 'All', 'Alluring', 'Almost', 'Also', 'Am', 'Ambroise', 'America', 'American', 'An', 'And', 'Angeles', 'Animals', 'Another', 'Anu', 'Any', 'Anyhow', 'Anyone', 'Arabian', 'Are', 'Armies', 'Army', 'Around', 'As', 'Assorted', 'At', 'Athletic', 'Athletics', 'Atlantis', 'Aunt', 'Australia', 'Away', 'Awful', 'B', 'BAUM', 'BEARS', 'BEING', 'BELT', 'BLACK', 'BOOKS', 'BOY', 'BRAIDED', 'BUGGY', 'BY', 'Bah', 'Bailum', 'Band', 'Banner', 'Barney', 'Baum', 'Be', 'Bears', 'Beasts', 'Because', 'Before', 'Behind', 'Behold', 'Being', 'Below', 'Belt', 'Beside

In [22]:
# string_to_int = { ch:i for i,ch in enumerate(chars) }
# int_to_string = { i:ch for i,ch in enumerate(chars) }
# encode = lambda s: [string_to_int[c] for c in s]
# decode = lambda l: ' '.join([int_to_string[i] for i in l])

# data = torch.tensor(encode(text), dtype=torch.long)
# print(data[:100])

In [23]:
# Get unique tokens and create mapping dictionaries
string_to_int = {ch: i for i, ch in enumerate(unique_tokens)}
int_to_string = {i: ch for i, ch in enumerate(unique_tokens)}

# Function to encode a sequence of tokens into integers
encode = lambda s: [string_to_int[c] for c in re.findall(r'\w+|[^\w\s]', s, re.UNICODE)]

# Function to decode a sequence of integers back into tokens
decode = lambda l: ''.join([int_to_string[i] + ' ' for i in l]).strip()

# Encode the entire text
# Encode the text using the updated mappings
data = torch.tensor([string_to_int[token] for token in unique_tokens], dtype=torch.long)
print(data[:100])

# Example of decoding to verify correctness
decoded_text = decode(data.tolist()[:100])
print(decoded_text)

tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
        36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53,
        54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71,
        72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89,
        90, 91, 92, 93, 94, 95, 96, 97, 98, 99])
! " & ' ( ) * , - . 1 10 106 11 12 120 13 131 14 142 15 16 160 17 172 18 187 19 1908 2 20 203 217 23 231 240 251 3 4 41 5 55 6 64 7 77 8 88 9 95 : ; ? A AGAIN AIR ALL AND ANOTHER ARE ARRIVAL AUTHOR About Accuser Adjustable After Afterward Ah Ahem All Alluring Almost Also Am Ambroise America American An And Angeles Animals Another Anu Any Anyhow Anyone Arabian Are Armies Army Around As Assorted At Athletic Athletics Atlantis Aunt Australia Away


In [24]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]


def get_random_chunk(split):
    filename = "train_split.txt" if split == 'train' else "val_split.txt"
    with open(filename, 'rb') as f:
        with mmap.mmap(f.fileno(), 0, access=mmap.ACCESS_READ) as mm:
            # Determine the file size and a random position to start reading
            file_size = len(mm)
            start_pos = random.randint(0, (file_size) - block_size*batch_size)

            # Seek to the random position and read the block of text
            mm.seek(start_pos)
            block = mm.read(block_size*batch_size-1)

            # Decode the block to a string, ignoring any invalid byte sequences
            decoded_block = block.decode('utf-8', errors='ignore').replace('\r', '')
            
            # Train and test splits
            data = torch.tensor(encode(decoded_block), dtype=torch.long)
            
    return data


def get_batch(split):
    # data = train_data if split == 'train' else val_data
    # ix = torch.randint(len(data) - block_size, (batch_size,))
    # # print(ix)
    # x = torch.stack([data[i:i+block_size] for i in ix])
    # y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    # x, y = x.to(device), y.to(device)
    # return x, y


    data_size = len(data)
    ix = torch.randint(data_size - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])

    # Ensure x and y are within valid range and have no NaNs
    # assert x.max() < vocab_size, f"x contains out-of-bounds indices: {x.max()} >= {vocab_size}"
    # assert y.max() < vocab_size, f"y contains out-of-bounds indices: {y.max()} >= {vocab_size}"
    # assert not torch.isnan(x).any(), "x contains NaNs"
    # assert not torch.isnan(y).any(), "y contains NaNs"

    x, y = x.to(device), y.to(device)
    return x, y

    

x, y = get_batch('train')
print('inputs:')
print(x.shape)
print(x)


print('targets:')
print(y)

inputs:
torch.Size([32, 128])
tensor([[ 388,  389,  390,  ...,  513,  514,  515],
        [3179, 3180, 3181,  ..., 3304, 3305, 3306],
        [1134, 1135, 1136,  ..., 1259, 1260, 1261],
        ...,
        [1824, 1825, 1826,  ..., 1949, 1950, 1951],
        [1986, 1987, 1988,  ..., 2111, 2112, 2113],
        [4241, 4242, 4243,  ..., 4366, 4367, 4368]], device='cuda:0')
targets:
tensor([[ 389,  390,  391,  ...,  514,  515,  516],
        [3180, 3181, 3182,  ..., 3305, 3306, 3307],
        [1135, 1136, 1137,  ..., 1260, 1261, 1262],
        ...,
        [1825, 1826, 1827,  ..., 1950, 1951, 1952],
        [1987, 1988, 1989,  ..., 2112, 2113, 2114],
        [4242, 4243, 4244,  ..., 4367, 4368, 4369]], device='cuda:0')


In [28]:
<<<<<<< Tabnine <<<<<<<
class Head(nn.Module):#-
    """ one head of self-attention """#-
def __init__(self, head_size):#+
    """#+
    Initializes a single head of self-attention.#+

    def __init__(self, head_size):#-
        super().__init__()#-
        self.key = nn.Linear(n_embd, head_size, bias=False)#-
        self.query = nn.Linear(n_embd, head_size, bias=False)#-
        self.value = nn.Linear(n_embd, head_size, bias=False)#-
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))#-
    Parameters:#+
    head_size (int): The size of each head. This determines the dimensionality of the linear layers' outputs.#+

        self.dropout = nn.Dropout(dropout)#-
    Attributes:#+
    key (nn.Linear): The linear layer for computing the keys.#+
    query (nn.Linear): The linear layer for computing the queries.#+
    value (nn.Linear): The linear layer for computing the values.#+
    tril (torch.Tensor): A lower triangular matrix used for masking out future positions in the sequence.#+
    dropout (nn.Dropout): The dropout layer for regularization.#+
    """#+
    super().__init__()#+
    self.key = nn.Linear(n_embd, head_size, bias=False)#+
    self.query = nn.Linear(n_embd, head_size, bias=False)#+
    self.value = nn.Linear(n_embd, head_size, bias=False)#+
    self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))#+
#+
    self.dropout = nn.Dropout(dropout)#+
>>>>>>> Tabnine >>>>>>>


def forward(self, x):
        
        """
        Performs the forward pass of the self-attention mechanism.
    
        Parameters:
        x (torch.Tensor): Input tensor of shape (batch, time-step, channels).
    
        Returns:
        torch.Tensor: Output tensor of shape (batch, time-step, head size).
        """
        B,T,C = x.shape
        k = self.key(x)   # (B,T,hs)
        q = self.query(x) # (B,T,hs)
    
        # Compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5 # (B, T, hs) @ (B, hs, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
    
        wei = self.dropout(wei)
    
        # Perform the weighted aggregation of the values
        v = self.value(x) # (B,T,hs)
        out = wei @ v # (B, T, T) @ (B, T, hs) -> (B, T, hs)
    
        return out


# [1, 0, 0]
# [1, 0.6, 0]
# [1, 0.6, 0.4]
class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1) # (B, T, F) -> (B, T, [h1, h1, h1, h1, h2, h2, h2, h2, h3, h3, h3, h3])
        out = self.dropout(self.proj(out))
        return out
    

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)
    
class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        y = self.sa(x)
        x = self.ln1(x + y)
        y = self.ffwd(x)
        x = self.ln2(x + y)
        return x
    
class GPTLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)
        
        
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, index, targets=None):
        B, T = index.shape
        
        
        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(index) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            index_cond = index[:, -block_size:]
            # get the predictions
            logits, loss = self.forward(index_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index

model = GPTLanguageModel(vocab_size)
print('loading model parameters...')
with open('model-01.pkl', 'rb') as f:
    model = pickle.load(f)
print('loaded successfully!')
m = model.to(device)

loading model parameters...
loaded successfully!


g:\PROJECT_WORKS\fcc-gpt\fcc-gpt-cuda\Lib\site-packages\torch\storage.py:414: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(io.BytesIO(b))


In [13]:
"""
Estimates the loss of the model on the training and validation datasets.

This function evaluates the model's performance on both the training and validation datasets.
It does so by iterating over a specified number of iterations (eval_iters) for each dataset.
During each iteration, it samples a batch of data from the respective dataset,
forward passes the batch through the model, and computes the loss.
The computed losses for each iteration are then stored in a tensor.
Finally, the mean of the losses for each dataset is calculated and returned as a dictionary.

Parameters:
None

Returns:
out (dict): A dictionary containing the mean loss for the training and validation datasets.
            The keys of the dictionary are 'train' and 'val', corresponding to the training and validation datasets, respectively.
"""
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out


In [26]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    print(iter)
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

with open('model-01.pkl', 'wb') as f:
    pickle.dump(model, f)
print('model saved')

0
step: 0, train loss: 8.501, val loss: 8.502
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
step: 250, train loss: 0.082, val loss: 0.082
251
252
253
254


In [27]:
prompt = 'Hello! Can you see me?'
context = torch.tensor(encode(prompt), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context.unsqueeze(0), max_new_tokens=100)[0].tolist())
print(generated_chars)

Hello ! Can you see me ? A AGAIN AIR ALL AND ANOTHER ARE ARRIVAL AUTHOR About Accuser Adjustable After Afterward Ah Ahem All Alluring Almost Also Am Ambroise America American An And Angeles Animals Another Anu Any Anyhow Anyone Arabian Are Armies Army Around As Assorted At Athletic Athletics Atlantis Aunt Australia Away Awful B BAUM BEARS BEING BELT BLACK BOOKS BOY BRAIDED BUGGY BY Bah Bailum Band Banner Barney Baum Be Bears Beasts Because Before Behind Behold Being Below Belt Besides Bill Billina Black Blackness Book Breakfast Bug Buggy Bush But By CAB CHAPTER CHAPTERS CITY CLOUD CO COME COPYRIGHT CORONADO COURT CUT Cab California


In [15]:

class GPTLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)
        
    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)
        
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self.forward(index)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index

model = GPTLanguageModel(vocab_size)
m = model.to(device)
# m = model

context = torch.zeros((1,1), dtype=torch.long, device=device)
# context = torch.zeros((1,1), dtype=torch.long)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)

! scatter clouds nickle ladies stubborn puffed admiration Gid jewelled articles murderous amazed discover ful_ intruder engulfed indeed sake uninvited uniform flowing scare Wizard poor picturesque adventure broke pulling didn ESCAPING cottage secluded swiftness lately seated crust attached Because wriggled sorcery entire bent dark Wait temper likely VALLEY punished pulled neared represented balance jury apartment lights Once bodies Wicked Now KINGDOM trembling stared almost scratching criss softest titles Chapter valley since attend qualities therefore Folks 203 likes grandfather journey farewell wherein bank stature sniff bonfire scampered EXCLAIMED startled _me_ day set yelled Country mouthful unprepared friend Tastes Witch Noticing untied praise sense obliged tenderly Pit CITY pleasantly offered nibble handed knowledge properly love scratched shiver contemptuous renew stiff Winkies Respected trunks rival scatter attractively contented MEET material fortune Athletic dinner thoughtful

In [16]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

step: 0, train loss: 8.963, val loss: 8.965
step: 250, train loss: 8.863, val loss: 8.864
step: 500, train loss: 8.766, val loss: 8.766
step: 750, train loss: 8.667, val loss: 8.669
8.591064453125


In [17]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)

! Respected yours blues blades bees true private onto reproachfully destroyed chins Fly puzzling flight Kingdom to tub heaped dearly seem swiftness traceries scuttles overhead exercise struggle tame Mr changing curving pets abandon sad taking Thought commit importance matted criticise ribbon parting mocking turned enable total split extreme unusual scorn destroyed humbly suit substance glowing In leads sleeves deny heavy rubbed shed driving awkward weeds flags clear Australia unusual regretful jaws Rain late pursuit Princess bit scales various From instead charging answer station clothed Prisoner Prince struck pausing producing balloon nodded BELT Ah ambition marble mounting longs Nome resume S tight DANGEROUS shutting maiden MOUNTAIN at _went_ became machinery combed talking assemblage Permit yelling illuminated jewel sank breath picking pert winging boards murdering _go_ twinkling advanced Having yards wailed lighter MANGABOOS another sits intruder make believe disagreeable bore fine